In [14]:
import os
import pandas as pd

from utils.makepath import makepath

In [15]:
root_dir = makepath("..", "..")

os.listdir(root_dir)

['scripts',
 'requirements.txt',
 'mri.egg-info',
 'venv',
 'README.md',
 'pretrained',
 'figures',
 'config',
 'LICENSE',
 'notebooks',
 'utils',
 'networks',
 'tmp',
 '.gitignore',
 'train_mri_example.ipynb',
 'gradops',
 'pyproject.toml',
 'results',
 'data',
 'pdhg',
 'model_10_20-15_48.pt',
 '.gitattributes',
 'model_10_20-15_47.pt',
 'wandb',
 'encoding_objects',
 '.git',
 'model_10_20-15_27.pt']

In [16]:
results_path = makepath(root_dir, "results", "test_set")

os.listdir(results_path)

['scalar_tgv', 'u_tgv', 'u_tv', 'zero_filled']

In [22]:
# model = "scalar_tgv"
model = "scalar_tv"
best_metric = "SSIM"
for acc_R in [4, 8]:
    for sigma in ["0_05", "0_1", "0_2"]:
        # acc_R, sigma, datetime = 4, "0_05", "21_01_59"
        # acc_R, sigma, datetime = 4, "0_1", "21_03_20"
        # csv_file = f"test-abs-scalar_tgv-best_SSIM-R_{acc_R}-sigma_{sigma}-lambda0_w_0_to_0_2-lambda1_v_0_to_0_2-ternary_iters_20-dt_{datetime}.csv"
        for file in os.listdir(makepath(results_path, model)):
            if f"test-abs-{model}-best_{best_metric}-R_{acc_R}-sigma_{sigma}" in file and file.endswith(".csv"):
                csv_file = file
                break
        df = pd.read_csv(makepath(results_path, model, csv_file))
        # print(df.head())
        print(f"R = {acc_R}, sigma = {sigma}")
        print(f"PSNR: {df['PSNR'].mean():.2f} ± {df['PSNR'].std():.2f}")
        print(f"SSIM: {df['SSIM'].mean():.4f} ± {df['SSIM'].std():.3f}")
        print()

# model = "u_tv"
# csv_file = makepath(
#     "test-abs-R_4-sigma_0_05",
#     "test-abs-R_4-sigma_0_05_intermediate_metrics.csv"
# )

R = 4, sigma = 0_05
PSNR: 27.21 ± 1.81
SSIM: 0.7134 ± 0.088

R = 4, sigma = 0_1
PSNR: 27.15 ± 1.82
SSIM: 0.7144 ± 0.090

R = 4, sigma = 0_2
PSNR: 26.91 ± 1.79
SSIM: 0.7096 ± 0.088

R = 8, sigma = 0_05
PSNR: 23.92 ± 1.71
SSIM: 0.6262 ± 0.085

R = 8, sigma = 0_1
PSNR: 23.87 ± 1.71
SSIM: 0.6264 ± 0.085

R = 8, sigma = 0_2
PSNR: 23.77 ± 1.71
SSIM: 0.6249 ± 0.085



In [15]:
csv_path = makepath(results_path, model, csv_file)

df = pd.read_csv(csv_path)
df.head()

,id,PSNR,SSIM,lambda1_v,lambda0_w
0,1,29.207605,0.817631,0.010632,0.016012
1,2,29.169151,0.738673,0.003013,0.066697
2,3,29.936134,0.680358,0.045261,0.046436
3,4,27.751083,0.805316,0.009505,0.015868
4,5,25.518986,0.756423,0.017461,0.040162


In [16]:
df.mean()

id           151.500000
PSNR          28.386008
SSIM           0.758968
lambda1_v      0.015164
lambda0_w      0.030659
dtype: float64

In [17]:
df.std()

id           87.324109
PSNR          2.115147
SSIM          0.063546
lambda1_v     0.020878
lambda0_w     0.023050
dtype: float64